In [1]:
import pandas as pd
import os
from pathlib import Path
from tqdm import tqdm
import json

## Load Perspective API scores and calculate MaxToxicity

In [2]:
def load_perspective_scores(output_dir, dataset_type):
    files = [i for i in os.listdir(output_dir) if i.startswith('perspective_api_all_occupations_all_gens') and i.endswith('.csv')]
    perspective_scores = pd.concat([pd.read_csv(os.path.join(output_dir,i)) for i in files])
    if 'dataset_type' not in perspective_scores.columns:
        perspective_scores = perspective_scores.assign(dataset_type = dataset_type)
    
    perspective_scores['model_name'] = perspective_scores['model_name'].apply(lambda x: x.split('/')[-1].replace('-hf',''))
    perspective_scores['prompt_original'] = perspective_scores['prompt']
    perspective_scores['prompt'] = perspective_scores['prompt'].apply(lambda x: '_'.join(x.split('_')[:-1]))
    perspective_scores = perspective_scores.assign(seed = perspective_scores['prompt_original'].apply(lambda x: x.split('_')[-1]))
    
    if 'dataset_type' in perspective_scores.columns:
        print(perspective_scores[['model_name','dataset_type']].value_counts())
    else:
        print(perspective_scores['model_name'].value_counts())

    return perspective_scores


def format_df_agg(df_agg, agg_cols, col, dataset, prompt):
    df_agg = df_agg.describe(percentiles=[i / 100 for i in range(5,100,5)]).transpose().iloc[1:,]
    df_agg = df_agg[agg_cols]
    df_agg.index = df_agg.index.rename('model')
    df_agg = df_agg.reset_index()
    df_agg = df_agg.assign(score = col, dataset = dataset, prompt = prompt)[['score','dataset', 'prompt', 'model'] + agg_cols]

    return df_agg


def calc_max_toxicity(perspective_scores):
    prompts = perspective_scores['prompt'].unique()
    datasets = perspective_scores['dataset_type'].unique()
    score_cols = ['TOXICITY']
    # score_cols = ['TOXICITY', 'INSULT','IDENTITY_ATTACK','FLIRTATION']
    agg_cols = ['mean', 'std', '25%', '50%', '75%', '90%', 'max']
    
    df_list = []
    df_diff_list = []
    df_max_id_list = []
    for col in tqdm(score_cols):
        for dataset in datasets:
            for prompt in prompts:
                df_agg = perspective_scores[(perspective_scores['prompt'] == prompt) & (perspective_scores['dataset_type'] == dataset)]
                df_agg = pd.pivot_table(df_agg, values=col, columns='model_name', index=['filename','rank','im_seed','im_index','prompt_original']).reset_index()
                df_agg_max = df_agg.groupby(['filename','rank','im_seed','prompt_original']).agg('max')
                df_agg_min = df_agg.groupby(['filename','rank','im_seed','prompt_original']).agg('min')
                df_agg_diff = df_agg_max - df_agg_min

                df_agg_id_max = df_agg.groupby(['filename','rank','im_seed','prompt_original']).idxmax()
                del df_agg_id_max['im_index']
                for i in df_agg_id_max.columns:
                    df_agg_id_max.loc[:,i] = df_agg_id_max[i].apply(lambda x: df_agg.iloc[int(x)]['im_index'])
                df_agg_id_max = df_agg_id_max.reset_index()
                df_agg_id_max = df_agg_id_max.assign(score = col, dataset = dataset, prompt = prompt)

                df_agg = format_df_agg(df_agg_max, agg_cols, col, dataset, prompt)
                df_agg_diff = format_df_agg(df_agg_diff, agg_cols, col, dataset, prompt)
                
                df_list.append(df_agg)
                df_diff_list.append(df_agg_diff)
                df_max_id_list.append(df_agg_id_max)

    return [pd.concat(df_list), pd.concat(df_diff_list), pd.concat(df_max_id_list)]

In [3]:
perspective_scores = load_perspective_scores('output/', 'race_gender')

model_name    dataset_type
llava-1.5-7b  race_gender     11526
Name: count, dtype: int64


In [4]:
max_toxicity, max_toxicity_diff, max_toxicity_id = calc_max_toxicity(perspective_scores)

100%|██████████| 1/1 [00:00<00:00,  7.41it/s]


In [5]:
max_toxicity

,score,dataset,prompt,model,mean,std,25%,50%,75%,90%,max
0,TOXICITY,race_gender,keywords_characteristics,llava-1.5-7b,0.139273,0.071592,0.091913,0.113987,0.178917,0.254629,0.426917


## Count frequency of competency words

In [6]:
generations = pd.read_csv('output/generations.csv')
generations = generations.assign(text_tokens = generations['keywords_characteristics_0'].str.lower().str.replace(',','').str.replace('.','').str.split())

In [7]:
competence_tokens = []
with open('competency_words.txt', 'r') as f:
    for line in f:
        competence_tokens.append(line.replace('\n',''))

In [8]:
generations = generations.assign(competence_words = generations.apply(lambda x: set(x['text_tokens']).intersection(competence_tokens), axis=1))
generations = generations.assign(competence_count = generations['competence_words'].str.len())

In [9]:
generations['competence_count'].describe()

count    11526.000000
mean         0.183498
std          0.432207
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max          4.000000
Name: competence_count, dtype: float64